In [ ]:
# import base packages for preprocessing shape files
import geopandas as gpd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import contextily as cx
import mapclassify
import folium

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

# remove scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# by including this autoreload command, we only need to re-import Spatial_Joins if we make/save changes to the original py file
%autoreload
from functions.partial_assignment import partial_assignment

### Import Data

In [ ]:
red = gpd.read_file("data/fullDownload.geojson")
sab = gpd.read_file("data/temm_v2.geojson")

### Pre-Process Geospatial Data

#### Redlining

In [ ]:
print(red.shape)
red.sample(5)

In [ ]:
# drop empty geometries, reset index
red = red.loc[red.is_valid]
red.reset_index(inplace=True, drop=True)

# set projection for spatial merging
red.to_crs(epsg=4269)

#### Service Area Bounds

In [ ]:
# drop empty geometries, reset index
sab = sab.loc[sab.is_valid]
sab.reset_index(inplace=True, drop=True)

In [ ]:
#subset the water system shape files to reduced columns and only Tier 1 data
sab = sab[['pwsid', 'pws_name', 'state_code', 'city_served', 'county_served', 'population_served_count', 
           'service_connections_count', 'tier', 'geometry']].copy()

sab = sab.query("tier=='Tier 1'").copy()

### Merge Redlining & Service Area Bounds Data

In [ ]:
# call the area overlap function to determine where HOLC graded geographies overlap with water systems
# and store the size of the intersection and the extent to which they overlap
red_sab = partial_assignment.calculate_area_percent(source_sf=red, 
                                                    target_sf=sab, 
                                                    target_sf_uid='pwsid', 
                                                    source_sf_uid='neighborhood_id')

In [ ]:
# drop tetxt column and geometry for storing values as csv
red_sab.drop(['area_description_data', 'geometry'], axis=1, inplace=True)
red_sab.drop_duplicates(inplace=True, ignore_index=True)
red_sab.shape

In [ ]:
# create simple df of data for use in analysis
# red_sab.to_csv("data/output/red_sab.csv", index=False)

In [ ]:
red_sab.sample(5)

### Additional Data Prep - SAB Details

In [ ]:
# create simple export of service area bounds data for merging into final project data
sab_df = sab[['pwsid', 'pws_name', 'state_code', 'population_served_count']].copy()
# sab_df.to_csv("data/output/pws_by_state.csv", index=False)

### Additional Data Prep - Texas Interactive Map

In [ ]:
# repeat similar process as above but with smaller dataset to be more efficient for interactive mapping

#preprocessing redlining data
red = gpd.read_file("data/fullDownload.geojson")
red.to_crs(epsg=4269)
red = red.loc[red.is_valid]
red.reset_index(inplace=True, drop=True)

# preprocessing water system data
sab = gpd.read_file("data/temm_v2.geojson")
sab.to_crs(epsg=4269)
# drop empty geometries, reset index
sab = sab.loc[sab.is_valid]
sab.reset_index(inplace=True, drop=True)

# keep only texas water systems and neighborhoods
red_ex = red.query("state=='TX'").copy()
sab_ex = sab.query("state_code=='TX'").copy()

In [ ]:
# create new ordinal version of HOLC grade for testing mapping with Folium
red_ex.drop(columns=['holc_id','name'], inplace=True)
red_ex['holc_grade_num'] = [ord(holc_grade) for holc_grade in red_ex.holc_grade]

sab_ex.reset_index(inplace=True, drop=True)
red_ex.reset_index(inplace=True, drop=True)

# 
red_sab_ex = partial_assignment.calculate_area_percent(source_sf=red_ex, 
                                                       target_sf=sab_ex, 
                                                       target_sf_uid='pwsid', 
                                                       source_sf_uid='neighborhood_id')

In [ ]:
# drop most columns
red_sab_ex.drop(columns=['primacy_agency_code', 'city_served', 'state_code', 'neighborhood_id', 'state',
       'city', 'holc_grade', 'holc_grade_num', 'percent_overlap',
       'county_served', 'owner_type_code', 'is_wholesaler_ind', 'primacy_type',
       'primary_source_code', 'tier', 'centroid_lat', 'centroid_lon',
       'centroid_quality', 'pred_05', 'pred_50', 'pred_95',
       'matched_bound_geoid', 'matched_bound_name', 'area_description_data', 'intersection_size'], inplace=True)

# drop duplicates, store as GPD, then create new unique ID based on combination of other unique IDs
red_sab.drop_duplicates(inplace=True)
red_sab = gpd.GeoDataFrame(red_sab)
red_sab['intersection_id'] = str(red_sab.pwsid) + "_" + str(red_sab.neighborhood_id)

### Test Mapping

In [ ]:
# Test mapping before exporting files

# Initialize the basemap for contextual location
m = folium.Map(location=[30.5, -98.5], min_lon=-107, max_lon=-93, min_lat=29, max_lat=36, tiles=None)

# Create choropleth map
ch1 = folium.Choropleth(
            name='Service Area Bounds',
            geo_data=red_sab,
            data=red_sab,
            columns=['intersection_id', 'holc_grade_num'],
            key_on='feature.properties.intersection_id',
            bins=4,
            fill_color='Purples',
            fill_opacity=0.6, 
            line_opacity=0.3,
            highlight=True, 
            line_color='black').geojson.add_to(m)

# Create the tooltip for additional details
ch1.add_child(
    folium.features.GeoJsonPopup(['pws_name', 'holc_grade'], labels=True, 
                                   aliases=['Name', 'HOLC Grade'])
)

# Create choropleth map
ch2 = folium.Choropleth(
            name='Redlining HOLC Grades',
            overlay=False,
            geo_data=red_ex, 
            data=red_ex,
            columns=['neighborhood_id', 'holc_grade_num'],
            key_on='feature.properties.neighborhood_id',
            bins=4,
            fill_color='Reds',
            fill_opacity=0.6, 
            line_opacity=0.3,
            highlight=True,
            line_color='black').geojson.add_to(m)

# Create the on-click popup for additional details for each block group
ch2.add_child(
    folium.features.GeoJsonPopup(['city', 'holc_grade'], labels=True, 
                                   aliases=['City', 'HOLC Grade'])
)

# add basemap overlay
folium.TileLayer('cartodbpositron', max_bounds=True, zoom_start=7, min_zoom=7, 
                min_lon=-107, max_lon=-93, min_lat=29, max_lat=36,overlay=True,name="Basemap").add_to(m)

# add layer controls to map, display by default
folium.LayerControl(collapsed=False).add_to(m)

#display the map layers
m

### Export Interactive Map Files

In [ ]:
# Path hack
import os
# change directory from the current analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at the top level project folder
print(os.getcwd())

# save shapefiles for use in app
# red_sab.to_file("data/shapefiles/overlap_sab.shp")
# red_ex.to_file("data/shapefiles/redline_tx.shp")